In [2]:
import igraph as ig
import plotly.plotly as py
from plotly.graph_objs import *
from sequence.sequence_manager import *
from sequence.sequence_plotter import *

<div style="text-align:center"><h1> RANDOM-WALK GRAPH </h1></div>

In [3]:
rw_path = "/home/chris/Scrivania/url2vec/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalk.depth.100000.seqLen.10/"
rw_map_path = rw_path + "sequencesMapUrl.txt"
rw_seq_path = rw_path + "sequencesIDs.txt"

rw_sequence_map = get_sequence_map(rw_map_path)
rw_sequences = get_sequences(rw_seq_path, 1)

rw_graph = create_graph(rw_sequences, rw_sequence_map)
print(len(rw_graph.vs), len(rw_graph.es))

(760, 16043)


In [6]:
rw_sequence_map = get_sequence_map(rw_map_path)
rw_seq_map = get_seq_map(rw_map_path)
# for key in rw_sequence_map:
    # print rw_sequence_map[key] == rw_seq_map[key]
print len(rw_sequence_map), len(rw_seq_map)

760 760


In [4]:
rw_fig = gplot(rw_graph, "Random walk network")
py.iplot(rw_fig, filename="Random walk network")

/home/chris/anaconda3/envs/python2/lib/python2.7/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



<div style="text-align:center"><h1> RANDOM-WALK-FROM-HOMEPAGE GRAPH </h1></div>

In [5]:
rwfh_path = "/home/chris/Scrivania/url2vec/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalkFromHomepage.depth.100000.seqLen.10/"
rwfh_map_path = rwfh_path + "sequencesMapUrl.txt"
rwfh_seq_path = rwfh_path + "sequencesIDs.txt"


rwfh_sequence_map = get_sequence_map(rwfh_map_path)
rwfh_sequences = get_sequences(rwfh_seq_path, 1)

rwfh_graph = create_graph(rwfh_sequences, rwfh_sequence_map)
print len(rwfh_graph.vs), len(rwfh_graph.es)

(603, 6917)


In [34]:
rwfh_fig = gplot(rwfh_graph, "Random walk from Homepage network")
py.iplot(rwfh_fig, filename="Random walk from Homepage network")

<div style="text-align:center"><h1> RANDOM-WALK-WITH-LISTS GRAPH </h1></div>

In [6]:
rwwl_path = "/home/chris/Scrivania/url2vec/dataset/depth-100k/seqLen-10/cs.illinois.eduRandomWalkLists.depth.100000.seqLen.10/"
rwwl_map_path = rwwl_path + "sequencesMapUrl.txt"
rwwl_seq_path = rwwl_path + "sequencesIDs.txt"


rwwl_sequence_map = get_sequence_map(rwwl_map_path)
rwwl_sequences = get_sequences(rwwl_seq_path, 1)

rwwl_graph = create_graph(rwwl_sequences, rwwl_sequence_map)
print len(rwwl_graph.vs), len(rwwl_graph.es)

(908, 9214)


In [14]:
rwwl_fig = gplot(rwwl_graph, "Random walk with Lists network")
py.iplot(rwwl_fig, filename="Random walk with Lists network")

<div style="text-align:center"><h1> MANUALLY-COLORED-RWWL GRAPH </h1></div>

In [44]:
color_rwwl_path = rwwl_path + "sequencesMapUrl-manually-clusterized.txt"
rwwl_sequences_wc = get_sequences(rwwl_seq_path, 1)
rwwl_tuplist = get_sequence_tuple_list(color_rwwl_path)

rwwl_graph_wc = create_graph(rwwl_sequences_wc, rwwl_tuplist)
print len(rwwl_graph_wc.vs), len(rwwl_graph_wc.es)

908
9214


In [10]:
rwmc = gplot(rwwl_graph_wc, "RWWL network - Manually colored")
py.iplot(rwmc, filename="RWWL network - Manually colored")

<div style="text-align:center"><h1> CLUSTERING </h1></div>

In [8]:
vertex_dendogram = rwwl_graph.community_fastgreedy()
vertex_clustering = vertex_dendogram.as_clustering(8)

In [9]:
type(vertex_clustering.membership)

list

In [17]:
print vertex_clustering.sizes()
print vertex_clustering.modularity

[164, 308, 145, 93, 56, 78, 61, 3]
0.292873015219


In [18]:
# verde, azzurro, giallo, bianco, arancione, blu, fucsia, viola
colors = ["#76FF03", "#00E5FF", "#FFFF00", "#FFFFFF", "#FF8F00", "#2979FF", "#F50057", "#9C27B0"]

for i in range(len(rwwl_graph.vs)):
    rwwl_graph.vs[i]["color"] = colors[vertex_clustering.membership[i]]
    # print rw_graph.vs[i]["name"], vertex_clustering.membership[i]

In [19]:
rwwl_clust_fig = gplot(rwwl_graph, "Random walk with lists - Modularity clustering")
py.iplot(rwwl_clust_fig, filename="Random walk with lists - Modularity clustering")

<div style="text-align:center"><h1> SINGLE CLUSTER </h1></div>

In [24]:
second_clust_subgraph = vertex_clustering.subgraph(1)
second_clust_fig = gplot(second_clust_subgraph, "Single cluster")

py.iplot(second_clust_fig, filename="Single cluster")

In [26]:
for v in second_clust_subgraph.vs:
    print v["name"]

https://cs.illinois.edu/giving/
https://cs.illinois.edu/research/
https://cs.illinois.edu/faculty-positions/
https://cs.illinois.edu/staff-positions/
https://cs.illinois.edu/alumni/
https://cs.illinois.edu/directory/staff/
https://cs.illinois.edu/directory/offices-and-contacts/
https://cs.illinois.edu/research/artificial-intelligence/
https://cs.illinois.edu/research/bioinformatics-and-computational-biology/
https://cs.illinois.edu/research/systems-and-networking/
https://cs.illinois.edu/research/database-and-information-systems/
https://cs.illinois.edu/research/graphics-visualization-and-human-computer-interaction/
https://cs.illinois.edu/directory/profile/jch/
https://cs.illinois.edu/directory/profile/daf/
https://cs.illinois.edu/news/student-startup-lumenous-brings-projection-mapping-out-arena/
https://cs.illinois.edu/directory/profile/kirlik/
https://cs.illinois.edu/research/architecture-compilers-and-parallel-computing/
https://cs.illinois.edu/directory/profile/padua/
https://cs.i